In [1]:
from Operations import SparkInst
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType

In [3]:
spark_job=SparkInst("AWAS SYSTEM", 5, "violations")
stream_a=spark_job.attach_kafka_stream("a", "9092", "5 minutes")
stream_b=spark_job.attach_kafka_stream("b", "9092", "5 minutes")
stream_b=spark_job.attach_kafka_stream("c", "9092", "5 minutes")

7


In [ ]:


# Assuming SparkInst is your class managing the Spark session and streams
spark_job = SparkInst("AWAS SYSTEM", 5, "violations")

# Step 1: Attach your Kafka streams
stream_a = spark_job.attach_kafka_stream("a", "9092", "5 minutes")
stream_b = spark_job.attach_kafka_stream("b", "9092", "5 minutes")
stream_c = spark_job.attach_kafka_stream("c", "9092", "5 minutes")  # Fixed variable name from your snippet

# Step 2: Load Mongo data into Pandas, convert to Spark DF
import pandas as pd
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017")
db = client["fit3182_db"]
coll = db["vehicle"]

mongo_docs = list(coll.find())
df_pd = pd.DataFrame(mongo_docs)
if '_id' in df_pd.columns:
    df_pd.drop(columns=['_id'], inplace=True)
spark_df = spark_job.get_session().createDataFrame(df_pd)

# Step 3: Broadcast your speed limit map
speed_limit_map = {row['camera_id']: row['speed_limit'] for row in spark_df.select("camera_id", "speed_limit").collect()}
broadcast_map = spark_job.essentialData_broadcast(spark_df)

# Step 4: Define your UDF using broadcast variable
def mark_speeding(camera_id, speed):
    limit = broadcast_map.value.get(camera_id)
    if limit is None:
        return "UNKNOWN"
    return "INSTANT_VIOLATION" if speed > limit else None

speeding_udf = udf(mark_speeding, StringType())

# Step 5: Apply UDF to each streaming dataframe
def add_speed_flag(df):
    return df.withColumn("speed_flag", speeding_udf(col("camera_id"), col("speed")))

stream_a_flagged = add_speed_flag(stream_a)
stream_b_flagged = add_speed_flag(stream_b)
stream_c_flagged = add_speed_flag(stream_c)

stream_a_flagged.join()






In [ ]:
spark_job.essentialData_broadcast()